# Numero de Nusselt con PySR

In [1]:
## Importando Librerias 
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### Cargando datos

In [3]:
# importacion de DF
file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_25.txt'
df_n_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_53.txt'
df_n_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_74.txt'
df_n_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'D:\\CODES\\LIB_SR\\Dataset_coeficientes\\df_n_102.txt'
df_n_102 = pd.read_csv(file_path, delimiter=',')

In [5]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_n_train = pd.concat([df_n_25, df_n_74], ignore_index=True)

# separando entre x e y
y_train = df_n_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_train = df_n_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])


# definiendo conjunto de test

y_test = df_n_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_test = df_n_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])

# renombrando
X_train.rename(columns={'Rem':'Re'}, inplace=True)
X_test.rename(columns={'Rem':'Re'}, inplace=True)

### Entrenamiento

In [9]:
model = PySRRegressor(
    niterations=100,
    binary_operators=["+", "*", "pow"],
    unary_operators=[],
    population_size=200,
    verbosity=1,
    maxsize=12,
    parsimony=1e-3,
    select_k_features=3,
    progress=True
)

model.fit(X_train, y_train)

d:\env\lib\site-packages\pysr\sr.py:2582: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
d:\env\lib\site-packages\pysr\sr.py:2029: UserWarning: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (https://astroautomata.com/PySR/options/#batching). You should also reconsider if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form with symbolic regression. More datapoints will lower the search speed.
  warnings.warn(


Using features ['K' 'Re' 'prandtl']


d:\env\lib\site-packages\pysr\sr.py:73: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant or variable (1). For more tips, please see https://astroautomata.com/PySR/tuning/
  warnings.warn(
[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 0.000e+00
Head worker occupation: 0.0%
Progress: 0 / 1500 total iterations (0.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 6.490e+01
Head worker occupation: 0.0%
Progress: 1 / 1500 total iterations (0.067%)
Hall of Fame:
-------------

PySRRegressor.equations_ = [
	   pick     score                                           equation  \
	0        0.000000                                          58.268833   
	1        1.272364                                     Re ^ 0.5084612   
	2        0.251769                     (Re * 0.22974885) ^ 0.61715025   
	3  >>>>  0.577173                (Re ^ 0.5300617) + (-13.170971 * K)   
	4        0.155244  ((Re * (K ^ -0.31161624)) ^ 0.5290738) + -13.1...   
	5        0.008558  ((((K ^ -0.31161624) * Re) ^ 0.61647844) + -1....   
	
	         loss  complexity  
	0  477.439880           1  
	1   37.476387           3  
	2   22.650310           5  
	3    7.140826           7  
	4    5.234862           9  
	5    5.146023          11  
]

In [12]:
best_equation = model.equations_.iloc[4]['equation']
print(best_equation)

((Re * (K ^ -0.31161624)) ^ 0.5290738) + -13.164498


In [11]:
best_equation = model.equations_.iloc[5]['equation']
print(best_equation)

((((K ^ -0.31161624) * Re) ^ 0.61647844) + -1.8832289) * 0.40571597


In [10]:
best_equation = model.equations_.iloc[3]['equation']
print(best_equation)

(Re ^ 0.5300617) + (-13.170971 * K)


In [14]:
X_test

,K,Re,prandtl
0,0.6,2824.604561,0.709843
1,0.6,2824.604561,0.709843
2,0.6,2352.302465,0.709842
3,0.6,2000.632597,0.709842
4,0.6,1878.735604,0.709842
...,...,...,...
8605,1.2,9267.885175,0.707768
8606,1.2,8959.387980,0.707705
8607,1.2,8691.558442,0.707625
8608,1.2,8537.950633,0.707554


### Pruebas

In [8]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 7.789375264358578
R^2: 0.9839406692094749
